In [9]:
import tomo3dt
import napari
import numpy as np
import dask.array as da
from dask.delayed import delayed
import os 
import time
from napari_animation import Animation

path = '/home/tomopc5/Documents/Datasets/Simulations/ML'
path_animations = '/home/tomopc5/Documents/Animations'
samples = ["NC-1", "NC-2", "NC-3", "NC-4"]
fname =["NC-1-vol.vmf", "NC-2-vol.vmf", "NC-3-vol.vmf", "NC-4-vol.vmf"]
files  = [os.path.join(path,fname[i]) for i in range(len(fname))]



In [10]:
n=3
vmf = tomo3dt.Volume4D(files[n])
shape = (vmf.dimx,vmf.dimy,vmf.dimz)
nptype = np.float32
lazy_imread = delayed(vmf.read_vmf_record)
lazy_arrays = [lazy_imread(t) for t in vmf.times]
dask_arrays = [
    da.from_delayed(delayed_reader, shape=shape, dtype=nptype)
    for delayed_reader in lazy_arrays
]
# Stack into one large dask.array
stack = da.stack(dask_arrays, axis=0)
stack.shape  # (nfiles, nz, ny, nx)

viewer = napari.Viewer(ndisplay=3)
image = viewer.add_image(stack, scale=[1, 1, 1], contrast_limits=[0,1], multiscale=False)
image.depiction = 'volume'
image.rendering = 'attenuated_mip'
image.colormap = 'bop orange'
viewer.scale_bar.visible = True
viewer.scale_bar.ticks = False

viewer.dims.current_step = (7,0,0,0)




In [11]:

animation = Animation(viewer)
viewer.update_console({'animation': animation})
x = 0
for i in vmf.times:
    viewer.dims.current_step = (i,0,0,0)
    for j in range(4):
        viewer.camera.angles=(0,x,0)
        animation.capture_keyframe(steps=15)
        x += 90

animation.animate(os.path.join(path_animations, samples[n]+'-true-vol.mp4'),fps=30, quality=8 )

Rendering frames...


100%|██████████| 4306/4306 [03:41<00:00, 19.44it/s]
